In [13]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [14]:
pickle_file= 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save=pickle.load(f)
    train_dataset=save['train_dataset']
    train_labels=save['train_labels']
    valid_dataset=save['valid_dataset']
    valid_labels=save['valid_labels']
    test_dataset=save['test_dataset']
    test_labels=save['test_labels']
    del save #hint to help gc to free up memory 
    print('training set', train_dataset.shape,train_labels.shape)
    print('validation set', valid_dataset.shape, valid_labels.shape)
    print('test set', test_dataset.shape,test_labels.shape)
    

training set (200000, 28, 28) (200000,)
validation set (10000, 28, 28) (10000,)
test set (10000, 28, 28) (10000,)


In [15]:
image_size=28
num_labels=10

def reformat(dataset, labels):
    dataset=dataset.reshape((-1,image_size*image_size)).astype(np.float32)
    # Map 1 to [0.0,1.0,0.0....], 2 to [0.0,0.0,1.0.....]
    labels=(np.arange(num_labels) ==labels[:,None]).astype(np.float32)
    return dataset,labels
train_dataset, train_labels= reformat(train_dataset, train_labels)
valid_dataset, valid_labels=reformat(valid_dataset, valid_labels)
test_dataset, test_labels =reformat(test_dataset, test_labels)
print( 'training set', train_dataset.shape,train_labels.shape)
print('validation set', valid_dataset.shape,valid_labels.shape)
print('test set', test_dataset.shape,test_labels.shape)

training set (200000, 784) (200000, 10)
validation set (10000, 784) (10000, 10)
test set (10000, 784) (10000, 10)


In [16]:
def accuracy(predictions, labels):
    return(100.0*np.sum(np.argmax(predictions, 1)==np.argmax(labels,1))/ predictions.shape[0])

In [17]:
#L2 regularization for both logistic and neural network models
#with one hidden layer fully connected

In [18]:
log_dir='Hidden_1layer_regularize_log'

def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)
if tf.gfile.Exists(log_dir):
    tf.gfile.DeleteRecursively(log_dir)
    tf.gfile.MakeDirs(log_dir)

In [19]:
batch_size=128 #sgd minibatch size
num_nodes=1024 
SGDSTEP=0.5
beta=0.01 #regularizer constant
#NumberofLayers=2 #(front layer+1 hidden layer)

'''
def weight_variable(shape):
    initial=tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial:tf.constant(0.1, shape=shape)
        return tf.Variable(initial)
        '''
sess = tf.InteractiveSession()    
graph=tf.Graph()
with graph.as_default():
    # Input data.For the training data,we use a placeholder to fed with a training mini batch
    #at runtime with a training min i batch
    tf_train_dataset= tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    
    tf_train_labels=tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    with tf.name_scope('weights1'):
        weights_1=tf.Variable(tf.truncated_normal([image_size*image_size,num_nodes]))
        variable_summaries(weights_1)
    with tf.name_scope('biases1'):
        biases_1=tf.Variable(tf.zeros([num_nodes]))
        variable_summaries(biases_1)
    with tf.name_scope('weights2'):
        weights_2=tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
        variable_summaries(weights_2)
    with tf.name_scope('biases2'):
        biases_2=tf.Variable(tf.zeros(num_labels))
        variable_summaries(biases_2)
    with tf.name_scope('W1x_plus_b1'):
        preactivate = tf.matmul(tf_train_dataset,weights_1)+ biases_1
        tf.summary.histogram('pre_activations1', preactivate)
    relu_layer=tf.nn.relu(preactivate)
    tf.summary.histogram('activations', relu_layer)
    
    #training Computation.
    
    logits=tf.matmul(relu_layer,weights_2)+biases_2
    #loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    #loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    
    with tf.name_scope('cross_entropy'):
        # The raw formulation of cross-entropy,
        #
        # tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.softmax(y)),
        #                               reduction_indices=[1]))
        #
        # can be numerically unstable.
        #
        # So here we use tf.nn.softmax_cross_entropy_with_logits on the
        # raw outputs of the nn_layer above, and then average across
        # the batch.
        with tf.name_scope('total'):
            #normal loss function
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
            #loss function with l2 regularaization with beta 0.01
            regularizers=tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2)
            loss=tf.reduce_mean(loss+beta*regularizers)
    tf.summary.scalar('cross_entropy', loss)

    
    #optimizer
    with tf.name_scope('train'):
        optimizer=tf.train.GradientDescentOptimizer(SGDSTEP).minimize(loss)
    
    #predictions for training
    train_prediction=tf.nn.softmax(logits)
    valid_prediction=tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1)+biases_1),weights_2)+biases_2)
    test_predicton= tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weights_1)+biases_1),weights_2)+biases_2)
    
    
   
    
   
    print(graph)
    print(sess)

In [ ]:
num_steps=30001


 

with tf.Session(graph=graph) as sess:
    # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(log_dir + '/train',sess.graph)
    test_writer = tf.summary.FileWriter(log_dir + '/test')
    tf.global_variables_initializer().run()
    print("Initialized")
    c=0
    step_array=[]
    accuracy_array=[]
    
    for step in range(num_steps):
        #Pick an offset within taining data. which has been randomized.
        # Note: we could use better randomization across epochs
        offset= (step*batch_size) % (train_labels.shape[0] - batch_size) # we have used train_labels.shape[0] it is equal to the toatal no of image in the datasets
        
        #generate a minibatch , as SGD requires them
        batch_data= train_dataset[offset:(offset+batch_size),:]
        batch_labels= train_labels[offset:(offset+batch_size),:]
        
        #Prepare a dictionary telling the session where to feed the minibatch
        # The Key of the dictinary is the placeholder node of the graph to be fed,
        #and the value is the numpy array to feed to it.
        feed_dict={tf_train_dataset: batch_data,tf_train_labels: batch_labels}
        _,summary, l,predictions = sess.run( [optimizer,merged,loss, train_prediction], feed_dict=feed_dict)
        test_writer.add_summary(summary, step)
        
        if (step % 500==0):
            miniAccuracy=accuracy( valid_prediction.eval(), valid_labels)
            print("Minibatch loss at step %d: %f" % (step,l))
            print("Minibatch accurracy: %.1f%%" % miniAccuracy)
            print (c)
            step_array.append(step)
            accuracy_array.append(miniAccuracy)
            c=c+1
            plt.plot(step,miniAccuracy)
        print("Test Accuracy: %.1f%%" % accuracy(test_predicton.eval(),test_labels))
        
        ##print(accuracy(test_predicton.eval(),test_labels))
        
sess.close()

Initialized
Minibatch loss at step 0: 3450.922852
Minibatch accurracy: 25.5%
0
Test Accuracy: 28.0%
Test Accuracy: 49.5%
Test Accuracy: 48.1%
Test Accuracy: 55.9%
Test Accuracy: 66.6%
Test Accuracy: 73.7%
Test Accuracy: 77.9%
Test Accuracy: 78.9%
Test Accuracy: 77.5%
Test Accuracy: 82.2%
Test Accuracy: 78.5%
Test Accuracy: 81.4%
Test Accuracy: 81.5%
Test Accuracy: 83.0%
Test Accuracy: 82.7%
Test Accuracy: 80.9%
Test Accuracy: 82.5%
Test Accuracy: 78.4%
Test Accuracy: 74.9%
Test Accuracy: 77.1%
Test Accuracy: 78.3%
Test Accuracy: 80.2%
Test Accuracy: 76.3%
Test Accuracy: 78.2%
Test Accuracy: 81.7%
Test Accuracy: 80.5%
Test Accuracy: 81.5%
Test Accuracy: 80.5%
Test Accuracy: 80.2%
Test Accuracy: 81.4%
Test Accuracy: 83.8%
Test Accuracy: 82.2%
Test Accuracy: 83.6%
Test Accuracy: 83.2%
Test Accuracy: 82.3%
Test Accuracy: 80.8%
Test Accuracy: 79.6%
Test Accuracy: 85.4%
Test Accuracy: 86.5%
Test Accuracy: 84.5%
Test Accuracy: 85.5%
Test Accuracy: 84.9%
Test Accuracy: 80.2%
Test Accuracy: 85.

In [ ]:
#dropout
#problem3

In [ ]:
log_dir='Hidden_1layer_dropout_regularize_log'

def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)
if tf.gfile.Exists(log_dir):
    tf.gfile.DeleteRecursively(log_dir)
    tf.gfile.MakeDirs(log_dir)

In [ ]:
batch_size=128 #sgd minibatch size
num_nodes=1024 
SGDSTEP=0.5
beta=0.01 #regularizer constant/learning rate
#NumberofLayers=2 #(front layer+1 hidden layer)

'''
def weight_variable(shape):
    initial=tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial:tf.constant(0.1, shape=shape)
        return tf.Variable(initial)
        '''
sess = tf.InteractiveSession()    
graph=tf.Graph()
with graph.as_default():
    # Input data.For the training data,we use a placeholder to fed with a training mini batch
    #at runtime with a training min i batch
    tf_train_dataset= tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    
    tf_train_labels=tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    with tf.name_scope('weights1'):
        weights_1=tf.Variable(tf.truncated_normal([image_size*image_size,num_nodes]))
        variable_summaries(weights_1)
    with tf.name_scope('biases1'):
        biases_1=tf.Variable(tf.zeros([num_nodes]))
        variable_summaries(biases_1)
    with tf.name_scope('weights2'):
        weights_2=tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
        variable_summaries(weights_2)
    with tf.name_scope('biases2'):
        biases_2=tf.Variable(tf.zeros(num_labels))
        variable_summaries(biases_2)
    with tf.name_scope('W1x_plus_b1'):
        preactivate = tf.matmul(tf_train_dataset,weights_1)+ biases_1
        tf.summary.histogram('pre_activations1', preactivate)
    relu_layer=tf.nn.relu(preactivate)
    #dropuout
    drop_prob= tf.placeholder("float")
    relu_layer_dropout=tf.nn.dropout(relu_layer, drop_prob)
    tf.summary.histogram('activations', relu_layer_dropout)
    
    #training Computation.
    
    logits=tf.matmul(relu_layer_dropout,weights_2)+biases_2
    #loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    #loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    
    with tf.name_scope('cross_entropy'):
        # The raw formulation of cross-entropy,
        #
        # tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.softmax(y)),
        #                               reduction_indices=[1]))
        #
        # can be numerically unstable.
        #
        # So here we use tf.nn.softmax_cross_entropy_with_logits on the
        # raw outputs of the nn_layer above, and then average across
        # the batch.
        with tf.name_scope('total'):
            #normal loss function
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
            #loss function with l2 regularaization with beta 0.01
            regularizers=tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2)
            loss=tf.reduce_mean(loss+beta*regularizers)
    tf.summary.scalar('cross_entropy', loss)

    
    #optimizer
    with tf.name_scope('train'):
        optimizer=tf.train.GradientDescentOptimizer(SGDSTEP).minimize(loss)
    
    #predictions for training
    train_prediction=tf.nn.softmax(logits)
    valid_prediction=tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1)+biases_1),weights_2)+biases_2)
    test_predicton= tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weights_1)+biases_1),weights_2)+biases_2)
    
    
   
    
   
    print(graph)
    print(sess)

In [ ]:
num_steps=30001


 

with tf.Session(graph=graph) as sess:
    # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(log_dir + '/train',sess.graph)
    test_writer = tf.summary.FileWriter(log_dir + '/test')
    tf.global_variables_initializer().run()
    print("Initialized")
    c=0
    step_array=[]
    accuracy_array=[]
    
    for step in range(num_steps):
        #Pick an offset within taining data. which has been randomized.
        # Note: we could use better randomization across epochs
        offset= (step*batch_size) % (train_labels.shape[0] - batch_size) # we have used train_labels.shape[0] it is equal to the toatal no of image in the datasets
        
        #generate a minibatch , as SGD requires them
        batch_data= train_dataset[offset:(offset+batch_size),:]
        batch_labels= train_labels[offset:(offset+batch_size),:]
        
        #Prepare a dictionary telling the session where to feed the minibatch
        # The Key of the dictinary is the placeholder node of the graph to be fed,
        #and the value is the numpy array to feed to it.
        feed_dict={tf_train_dataset: batch_data,tf_train_labels: batch_labels,drop_prob: 0.5}
        _,summary, l,predictions = sess.run( [optimizer,merged,loss, train_prediction], feed_dict=feed_dict)
        test_writer.add_summary(summary, step)
        
        if (step % 500==0):
            miniAccuracy=accuracy( valid_prediction.eval(), valid_labels)
            print("Minibatch loss at step %d: %f" % (step,l))
            print("Minibatch accurracy: %.1f%%" % miniAccuracy)
            print (c)
            step_array.append(step)
            accuracy_array.append(miniAccuracy)
            c=c+1
            plt.plot(step,miniAccuracy)
        print("Test Accuracy: %.1f%%" % accuracy(test_predicton.eval(),test_labels))
        
        ##print(accuracy(test_predicton.eval(),test_labels))
        
sess.close()

In [ ]:
#3 logits, 2 hidden fully connected with regularization, dropout and learning decay

In [ ]:
log_dir='Hidden_2layer_dropout_regularize__decay_log'

def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)
if tf.gfile.Exists(log_dir):
    tf.gfile.DeleteRecursively(log_dir)
    tf.gfile.MakeDirs(log_dir)

In [ ]:
batch_size=128 #sgd minibatch size
num_nodes=1024 
hidden_nodes_2=int(num_nodes*0.5)
SGDSTEP=0.5
beta=0.01 #regularizer constant/learning rate
#NumberofLayers=2 #(front layer+1 hidden layer)

'''
def weight_variable(shape):
    initial=tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial:tf.constant(0.1, shape=shape)
        return tf.Variable(initial)
        '''
sess = tf.InteractiveSession()    
graph=tf.Graph()
with graph.as_default():
    # Input data.For the training data,we use a placeholder to fed with a training mini batch
    #at runtime with a training min i batch
    tf_train_dataset= tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    
    tf_train_labels=tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    #layer 1
    with tf.name_scope('weights1'):
        weights_1=tf.Variable(tf.truncated_normal([image_size*image_size,num_nodes]))
        variable_summaries(weights_1)
    with tf.name_scope('biases1'):
        biases_1=tf.Variable(tf.zeros([num_nodes]))
        variable_summaries(biases_1)
        
    #layer 2
    with tf.name_scope('weights2'):
        weights_2=tf.Variable(tf.truncated_normal([num_nodes, hidden_nodes_2]))
        variable_summaries(weights_2)
    with tf.name_scope('biases2'):
        biases_2=tf.Variable(tf.zeros(hidden_nodes_2))
        variable_summaries(biases_2)
    
    #output layer
    with tf.name_scope('weights3'):
        weights_3=tf.Variable(tf.truncated_normal([hidden_nodes_2,num_labels]))
        variable_summaries(weights_3)
    with tf.name_scope('biases3'):
        biases_3=tf.Variable(tf.zeros(num_labels))
        variable_summaries(biases_3)
        
    with tf.name_scope('W1x_plus_b1'):
        preactivate = tf.matmul(tf_train_dataset,weights_1)+ biases_1
        tf.summary.histogram('pre_activations1', preactivate)
    relu_layer_1=tf.nn.relu(preactivate)
    #dropuout
    drop_prob= tf.placeholder("float")
    relu_layer_1_dropout=tf.nn.dropout(relu_layer, drop_prob)
    tf.summary.histogram('activations', relu_layer_1_dropout)
    
    preactivate_2 = tf.matmul(tf_train_dataset,weights_2)+ biases_2
        tf.summary.histogram('pre_activations1', preactivate_2)
    relu_layer_2=tf.nn.relu(preactivate_2)
    #dropuout
    drop_prob= tf.placeholder("float")
    relu_layer_2_dropout=tf.nn.dropout(relu_layer, drop_prob)
    tf.summary.histogram('activations', relu_layer_2_dropout)
    
    #training Computation.
    
    logits=tf.matmul(relu_layer_2_dropout,weights_2)+biases_2
    #loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    #loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    
    with tf.name_scope('cross_entropy'):
        # The raw formulation of cross-entropy,
        #
        # tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.softmax(y)),
        #                               reduction_indices=[1]))
        #
        # can be numerically unstable.
        #
        # So here we use tf.nn.softmax_cross_entropy_with_logits on the
        # raw outputs of the nn_layer above, and then average across
        # the batch.
        with tf.name_scope('total'):
            #normal loss function
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
            #loss function with l2 regularaization with beta 0.01
            regularizers=tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2)+tf.nn.l2_loss(weights_3)
            loss=tf.reduce_mean(loss+beta*regularizers)
    tf.summary.scalar('cross_entropy', loss)

    
    #optimizer
    with tf.name_scope('train'):
        optimizer=tf.train.GradientDescentOptimizer(SGDSTEP).minimize(loss)
    
    #predictions for training
    train_prediction=tf.nn.softmax(logits)
    valid_prediction=tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1)+biases_1),weights_2)+biases_2)
    test_predicton= tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weights_1)+biases_1),weights_2)+biases_2)
    
    
   
    
   
    print(graph)
    print(sess)

In [ ]:
num_steps=30001


 

with tf.Session(graph=graph) as sess:
    # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(log_dir + '/train',sess.graph)
    test_writer = tf.summary.FileWriter(log_dir + '/test')
    tf.global_variables_initializer().run()
    print("Initialized")
    c=0
    step_array=[]
    accuracy_array=[]
    
    for step in range(num_steps):
        #Pick an offset within taining data. which has been randomized.
        # Note: we could use better randomization across epochs
        offset= (step*batch_size) % (train_labels.shape[0] - batch_size) # we have used train_labels.shape[0] it is equal to the toatal no of image in the datasets
        
        #generate a minibatch , as SGD requires them
        batch_data= train_dataset[offset:(offset+batch_size),:]
        batch_labels= train_labels[offset:(offset+batch_size),:]
        
        #Prepare a dictionary telling the session where to feed the minibatch
        # The Key of the dictinary is the placeholder node of the graph to be fed,
        #and the value is the numpy array to feed to it.
        feed_dict={tf_train_dataset: batch_data,tf_train_labels: batch_labels,drop_prob: 0.5}
        _,summary, l,predictions = sess.run( [optimizer,merged,loss, train_prediction], feed_dict=feed_dict)
        test_writer.add_summary(summary, step)
        
        if (step % 500==0):
            miniAccuracy=accuracy( valid_prediction.eval(), valid_labels)
            print("Minibatch loss at step %d: %f" % (step,l))
            print("Minibatch accurracy: %.1f%%" % miniAccuracy)
            print (c)
            step_array.append(step)
            accuracy_array.append(miniAccuracy)
            c=c+1
            plt.plot(step,miniAccuracy)
        print("Test Accuracy: %.1f%%" % accuracy(test_predicton.eval(),test_labels))
        
        ##print(accuracy(test_predicton.eval(),test_labels))
        
sess.close()